# NLP Task03 Jing-Long Wu / 3045999 

In [23]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
import re

In [24]:
#using lstm in model_1
def lstm():
    nn = Sequential()
    nn.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
    nn.add(SpatialDropout1D(dropout_1d))
    nn.add(LSTM(lstm_out, dropout=dropout, recurrent_dropout=recurrent_dropout))
    nn.add(Dense(2, activation='softmax'))
    nn.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    print(nn.summary())
    return nn
#using convolution network in model2
def conv():
    nn = Sequential()
    nn.add(Embedding(max_features, embed_dim, input_length= X.shape[1]))
    nn.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    nn.add(MaxPooling1D(pool_size=2))
    nn.add(Flatten())
    nn.add(Dense(250, activation='relu'))
    nn.add(Dense(2, activation='softmax'))
    nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(nn.summary())
    return nn
#evaluation function
def evaluation(nn):
    nn.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose=verbose)
    score, accuracy = nn.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("score: %.2f" % (score))
    print("acc: %.2f" % (accuracy))

    pos_cnt, neg_cnt, pos_ok, neg_ok = 0, 0, 0, 0
    for x in range(len(X_validate)):
        result = nn.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
        if np.argmax(result) == np.argmax(Y_validate[x]):
            if np.argmax(Y_validate[x]) == 0: neg_ok += 1
            else: pos_ok += 1
        if np.argmax(Y_validate[x]) == 0: neg_cnt += 1
        else: pos_cnt += 1

    print("pos_acc", pos_ok/pos_cnt*100, "%")
    print("neg_acc", neg_ok/neg_cnt*100, "%")

    X2 = ['what are u going to say about that? the truth, wassock?!']
    X2 = tok.texts_to_sequences(X2)
    X2 = pad_sequences(X2, maxlen=26, dtype='int32', value=0)
    print(X2)
    print(nn.predict(X2, batch_size=1, verbose = 2)[0])

In [25]:
# parameters 
max_features = 500
embed_dim = 128
lstm_out = 196
dropout = 0.1
dropout_1d = 0.4
recurrent_dropout = 0.1
random_state = 4222
validation_size = 1000
batch_size = 16
epochs=2
verbose= 2

In [26]:
# Read and preprocess data

df = pd.read_csv('dataset_sentiment.csv')
df = df[['text','sentiment']]

df = df[df.sentiment != "Neutral"]
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply(lambda x: x.replace('rt',' '))
df['text'] = df['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    
tok = Tokenizer(num_words=max_features, split=' ')
tok.fit_on_texts(df['text'].values)
X = tok.texts_to_sequences(df['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(df['sentiment']).values

In [27]:
# Split Dataset

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = random_state)

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

In [28]:
#model 1
nn = lstm()
evaluation(nn)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 26, 128)           64000     
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 26, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 394       
Total params: 319,194
Trainable params: 319,194
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
 - 24s - loss: 0.4395 - acc: 0.8154
Epoch 2/2
 - 22s - loss: 0.3662 - acc: 0.8426
score: 0.36
acc: 0.85
pos_acc 28.780487804878046 %
neg_acc 98.36477987421384 %
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  49  38
  310 189   4 146 

In [29]:
#model 2
nn2 = conv()
evaluation(nn2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 26, 128)           64000     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 26, 32)            12320     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 13, 32)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 416)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 250)               104250    
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 502       
Total params: 181,072
Trainable params: 181,072
Non-trainable params: 0
_________________________________________________________________
None

In [33]:
#generate meta_data
from rdflib import Namespace, Graph, Literal
from rdflib.namespace import FOAF, OWL, XSD, RDFS, DCTERMS, DOAP, DC, RDF


prov = Namespace('http://www.w3.org/ns/prov#')
dcat = Namespace('http://www.w3.org/ns/dcat#')
mexalgo = Namespace('http://mex.aksw.org/mex-algo#')
mexperf = Namespace('http://mex.aksw.org/mex-perf#')
mexcore = Namespace('http://mex.aksw.org/mex-core#')
this = Namespace('http://mex.aksw.org/examples/')

g = Graph()
# Create Binding
g.bind('dct',DCTERMS)
g.bind('owl',OWL)
g.bind('foaf',FOAF)
g.bind('xsd', XSD)
g.bind('rdfs', RDFS)
g.bind('doap', DOAP)
g.bind('dc', DC)
g.bind('prov', prov)
g.bind('dcat', dcat)
g.bind('mexalgo',mexalgo)
g.bind('mexperf',mexperf)
g.bind('mexcore',mexcore)
g.bind('this',this)

g.add((this.JLWU_task3,mexcore.Experiment, prov.Entity))
g.add((this.JLWU_task3,mexcore.ApplicationContext, prov.Entity))
g.add((this.JLWU_task3,DCTERMS.date, Literal('2018-05-29',datatype=XSD.date)))
g.add((this.JLWU_task3,FOAF.givenName, Literal('Jing-Long Wu')))

#Configuration-1
g.add((this.configuration1,mexcore.ExperimentConfiguration, prov.Entity))
g.add((this.configuration1,prov.used, this.model1))
g.add((this.configuration1,prov.wasStartedBy, this.JLWU_task3))

#Configuration-2
g.add((this.configuration2,mexcore.ExperimentConfiguration, prov.Entity))
g.add((this.configuration2,prov.used, this.model2))
g.add((this.configuration2,prov.wasStartedBy, this.JLWU_task3))

#hyperparam1
g.add((this.hyerparameter_model1,mexalgo.HyperParameterCollection,prov.Entity))
g.add((this.hyerparameter1,RDFS.label,Literal('HyperParameterCollection')))
g.add((this.hyerparameter_model1,prov.hadMember,this.hyerparameter11))


#hyperparam11
g.add((this.hyerparameter1,mexalgo.HyperParameter,prov.Entity))
g.add((this.hyerparameter1,RDFS.label, Literal('LSTM')))
g.add((this.hyerparameter1,DCTERMS.identifier, Literal('LSTM')))
g.add((this.hyerparameter1,prov.value, Literal('196',datatype=XSD.float)))



#hyperparam2
g.add((this.hyerparameter_model2,mexalgo.HyperParameterCollection,prov.Entity))
g.add((this.hyerparameter2,RDFS.label,Literal('HyperParameterCollection')))
g.add((this.hyerparameter_model2,prov.hadMember,this.hyerparameter2))

g.add((this.hyerparameter2,mexalgo.HyperParameter,prov.Entity))
g.add((this.hyerparameter2,RDFS.label, Literal('conv')))
g.add((this.hyerparameter2,DCTERMS.identifier, Literal('conv')))
g.add((this.hyerparameter2,prov.value, Literal('26,32',datatype=XSD.float)))

#measure1
g.add((this.execution1,mexcore.ExecutionOverall,prov.Entity))
g.add((this.execution1,prov.generated,this.performance_measures1))
g.add((this.execution1,prov.used,this.test))
g.add((this.execution1,prov.used,this.hyerparameter_model1))
g.add((this.execution1,prov.used,this.model1))

g.add((this.performance_measures1,mexcore.PerformanceMeasure,prov.Entity))
g.add((this.performance_measures1,mexperf.score,Literal('0.36',datatype=XSD.float)))
g.add((this.performance_measures1,mexperf.accuracy,Literal('0.85',datatype=XSD.float)))
g.add((this.performance_measures1,prov.wasGeneratedBy,this.execution1))

#measure2
g.add((this.execution2,mexcore.ExecutionOverall,prov.Entity))
g.add((this.execution2,prov.generated,this.performance_measures2))
g.add((this.execution2,prov.used,this.test))
g.add((this.execution2,prov.used,this.model2))

g.add((this.performance_measures2,mexcore.PerformanceMeasure,prov.Entity))
g.add((this.performance_measures2,mexperf.score,Literal('0.38',datatype=XSD.float)))
g.add((this.performance_measures2,mexperf.accuracy,Literal('0.85',datatype=XSD.float)))
g.add((this.performance_measures2,prov.wasGeneratedBy,this.execution2))

#Model
g.add((this.model1,mexalgo.Algorithm,prov.Entity))
g.add((this.model1,RDFS.label,Literal('Keras-LSTM-NET')))
g.add((this.model1,mexalgo.hasHyperParameter,this.hyerparameter1))

g.add((this.model2,mexalgo.Algorithm,prov.Entity))
g.add((this.model2,RDFS.label,Literal('Keras-CONV-NET')))
g.add((this.model2,mexalgo.hasHyperParameter,this.hyerparameter2))


with open('JingLongWu_task3_consequence_metadata.ttl','wb') as f:
    f.write(g.serialize(format='turtle'))

D:\Anaconda3\lib\site-packages\rdflib\term.py:1262: UserWarning: Serializing weird numerical rdflib.term.Literal('26,32', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float'))
  warnings.warn("Serializing weird numerical %r" % self)
